In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
# Load CSV File

csv_file = "auto_annotations.csv"
df = pd.read_csv(csv_file)

# Select annotated_image_name, fabric_types columns only
df = df[["annotated_image_name", "wear_types"]]

df.dropna()

In [ ]:
FEATURE_NAME = 'wear_types'

In [ ]:
# Custom Dataset
class SingleClassDataset(Dataset):
	def __init__(self, csv_file, root_dir, transform=None):
		self.data = pd.read_csv(csv_file)
		self.data = self.data[["annotated_image_name", FEATURE_NAME]]
		self.data.dropna(inplace=True)
		self.data.reset_index(drop=True, inplace=True)
		
		self.root_dir = root_dir
		self.transform = transform
		self.classes = {fabric: idx for idx, fabric in enumerate(self.data[FEATURE_NAME].unique())}
	
	def __len__(self):
		return len(self.data)
	
	def __getitem__(self, idx):
		img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
		image = Image.open(img_name).convert('RGB')
		label = self.classes[self.data.iloc[idx, 1]]
		
		if self.transform:
			image = self.transform(image)
		
		return image, label

In [ ]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
# # Dataset & Dataloader
# dataset = SingleClassDataset(csv_file='auto_annotations.csv', root_dir='../data/resized', transform=transform)

In [ ]:
# Dataset & Splitting
dataset = SingleClassDataset(csv_file='auto_annotations.csv', root_dir='../data/resized', transform=transform)

train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [ ]:
# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(dataset.classes))
model = model.to(device)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training Loop
epochs = 3
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    val_accuracy = correct / total
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_accuracy:.4f}")

In [ ]:
# Save Model
torch.save(model.state_dict(), f'{FEATURE_NAME}_classifier.pth')


In [ ]:
# Test Evaluation
model.eval()
correct = 0
total = 0
test_loss = 0.0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_accuracy = correct / total
print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.4f}")